# BERT4Rec + GNN Hybrid Experiments

**Bidirectional Transformer with Graph Neural Network Fusion**

---

## 🎯 Experiment Goal

This notebook compares **BERT4Rec** (standalone) with **BERT4Rec + GNN hybrids** (all fusion variants) to test whether adding GNN improves bidirectional transformers for sequential recommendation.

**Key Difference from Previous Experiments:**
- Previous hybrids used **SASRec** (unidirectional) + GNN
- These new hybrids use **BERT4Rec** (bidirectional) + GNN
- Bidirectional attention is more powerful → fairer comparison

---

## 🎓 Configuration

**Training Settings:**
- Max Epochs: 200 (with early stopping)
- Patience: 20
- Expected convergence: epoch 30-70
- Batch size: 256
- Learning rate: 0.001
- Model: d_model=64, n_heads=2, n_blocks=2, gnn_layers=2

**Models to Train (5 Total):**

1. ✅ **BERT4Rec** (baseline - bidirectional transformer)
2. ✅ **BERT4Rec + GNN (Fixed)** - α=0.5 fusion
3. ✅ **BERT4Rec + GNN (Discrete)** - bin-based fusion
4. ✅ **BERT4Rec + GNN (Learnable)** - learned bin weights
5. ✅ **BERT4Rec + GNN (Continuous)** - neural fusion function

**Time Estimate: ~6-8 hours total with GPU T4**

---

## 📋 Quick Start

1. Enable **GPU T4** accelerator (Runtime → Change runtime type)
2. Enable **Internet** access
3. Run cells sequentially
4. Download results at the end


## Step 1: Clone Repository

In [ ]:
# Clone repository
!git clone https://github.com/faroukq1/length-adaptive.git

# Change to project directory
%cd length-adaptive

# Verify structure
!ls -lh scripts/

print("\n✅ Repository cloned successfully!")

## Step 2: Install Dependencies

In [ ]:
# Install required packages quietly
!pip install -q torch-geometric tqdm scikit-learn pandas matplotlib seaborn scipy

print("✅ All dependencies installed successfully!")

## Step 3: Verify GPU

In [ ]:
# Check GPU availability
!python check_gpu.py

## Step 4: Prepare Data

Downloads MovieLens-1M and preprocesses if needed (2-3 minutes)

In [ ]:
import os

# Check if preprocessed data exists
data_file = 'data/ml-1m/processed/sequences.pkl'
graph_file = 'data/graphs/cooccurrence_graph.pkl'

print("="*70)
print("🔍 Checking Data Files")
print("="*70)

if os.path.exists(data_file):
    print(f"✅ Sequential data found: {data_file}")
    print(f"   Size: {os.path.getsize(data_file) / 1024 / 1024:.2f} MB")
else:
    print(f"❌ Sequential data NOT found: {data_file}")

if os.path.exists(graph_file):
    print(f"✅ Graph data found: {graph_file}")
    print(f"   Size: {os.path.getsize(graph_file) / 1024 / 1024:.2f} MB")
else:
    print(f"❌ Graph data NOT found: {graph_file}")

# Check raw data
raw_file = 'data/ml-1m/raw/ml-1m/ratings.dat'
if os.path.exists(raw_file):
    print(f"✅ Raw data found: {raw_file}")
else:
    print(f"❌ Raw data NOT found: {raw_file}")

print("="*70)

# If data is missing, run preprocessing
if not os.path.exists(data_file) or not os.path.exists(graph_file):
    print("\n🔧 Running preprocessing...")
    print("This will take 2-3 minutes.\n")
    
    # Download MovieLens-1M if needed
    if not os.path.exists(raw_file):
        print("📥 Downloading MovieLens-1M dataset...")
        !mkdir -p data/ml-1m/raw
        !wget -q http://files.grouplens.org/datasets/movielens/ml-1m.zip
        !unzip -q ml-1m.zip
        !mv ml-1m data/ml-1m/raw/
        !rm -f ml-1m.zip
        print("✅ Download complete!\n")
    
    # Run preprocessing
    print("🔄 Preprocessing sequential data...")
    !python -m src.data.preprocess
    
    # Build graph
    print("\n🔄 Building co-occurrence graph...")
    !python -m src.data.graph_builder
    
    print("\n✅ Preprocessing complete!")
    print("="*70)
else:
    print("\n✅ All data files ready!")
    print("="*70)

## Step 5: Run BERT4Rec + GNN Hybrid Experiments

**⏱️ Time: ~6-8 hours total (GPU T4)**

This will train BERT4Rec baseline + 4 hybrid variants sequentially with paper-quality settings:
- 200 max epochs with early stopping (patience=20)
- Models typically converge at epoch 30-70
- Fair comparison: all models use bidirectional attention
- Tests whether GNN improves BERT4Rec performance

**Models:**
1. BERT4Rec (baseline)
2. BERT4Rec + GNN (Fixed fusion, α=0.5)
3. BERT4Rec + GNN (Discrete bins)
4. BERT4Rec + GNN (Learnable bins)
5. BERT4Rec + GNN (Continuous neural fusion)


In [ ]:
# Run BERT hybrid experiments
print("="*80)
print("🎓 BERT4Rec + GNN HYBRID EXPERIMENTS")
print("="*80)
print("")
print("Training 5 models with 200 epochs, early stopping patience=20")
print("Expected convergence: epoch 30-70")
print("Time estimate: ~6-8 hours with GPU T4")
print("")
print("Baseline:")
print("  1. BERT4Rec (Bidirectional Transformer, CIKM 2019)")
print("")
print("Hybrid Models (Ours - BERT4Rec base):")
print("  2. BERT4Rec + GNN (Fixed fusion, α=0.5)")
print("  3. BERT4Rec + GNN (Discrete bins)")
print("  4. BERT4Rec + GNN (Learnable bins)")
print("  5. BERT4Rec + GNN (Continuous neural fusion)")
print("")
print("="*80)

# Run the BERT hybrid experiments script
!bash scripts/run_bert_hybrid_experiments.sh

print("\n✅ All BERT hybrid experiments complete!")

## Step 6: Generate Comparison Results

Create comparison tables specifically for BERT4Rec vs BERT+GNN hybrids

In [ ]:
# Generate comparison for BERT models
import os
import json
import pandas as pd
from pathlib import Path

print("="*70)
print("📊 Analyzing BERT4Rec vs BERT+GNN Hybrid Results")
print("="*70)

results_dir = Path('results')
models_to_compare = [
    'bert4rec',
    'bert_hybrid_fixed', 
    'bert_hybrid_discrete',
    'bert_hybrid_learnable',
    'bert_hybrid_continuous'
]

# Collect results
results_data = []
for result_folder in sorted(results_dir.glob('*')):
    if result_folder.is_dir():
        results_file = result_folder / 'results.json'
        config_file = result_folder / 'config.json'
        
        if results_file.exists() and config_file.exists():
            with open(results_file) as f:
                results = json.load(f)
            with open(config_file) as f:
                config = json.load(f)
            
            model_name = config['model']
            
            # Only include BERT models
            if model_name in models_to_compare:
                results_data.append({
                    'Model': model_name,
                    'HR@5': results['test_metrics']['HR@5'],
                    'HR@10': results['test_metrics']['HR@10'],
                    'HR@20': results['test_metrics']['HR@20'],
                    'NDCG@5': results['test_metrics']['NDCG@5'],
                    'NDCG@10': results['test_metrics']['NDCG@10'],
                    'NDCG@20': results['test_metrics']['NDCG@20'],
                    'MRR@5': results['test_metrics']['MRR@5'],
                    'MRR@10': results['test_metrics']['MRR@10'],
                    'MRR@20': results['test_metrics']['MRR@20'],
                    'Best Epoch': results['best_epoch'],
                    'Best Val NDCG@10': results['best_val_metric'],
                    'Short HR@10': results['grouped_metrics']['short']['HR@10'],
                    'Short NDCG@10': results['grouped_metrics']['short']['NDCG@10'],
                    'Medium HR@10': results['grouped_metrics']['medium']['HR@10'],
                    'Medium NDCG@10': results['grouped_metrics']['medium']['NDCG@10'],
                })

# Create DataFrame
df = pd.DataFrame(results_data)

# Sort by NDCG@10
df = df.sort_values('NDCG@10', ascending=False)

# Save overall comparison
overall_file = 'results/bert_hybrid_comparison_overall.csv'
df[['Model', 'HR@10', 'NDCG@10', 'MRR@10', 'Best Epoch', 'Best Val NDCG@10']].to_csv(
    overall_file, index=False
)
print(f"\n✅ Overall comparison saved: {overall_file}")

# Save short sequence comparison
short_file = 'results/bert_hybrid_comparison_short.csv'
df[['Model', 'Short HR@10', 'Short NDCG@10']].rename(
    columns={'Short HR@10': 'HR@10', 'Short NDCG@10': 'NDCG@10'}
).to_csv(short_file, index=False)
print(f"✅ Short sequence comparison saved: {short_file}")

# Save medium sequence comparison
medium_file = 'results/bert_hybrid_comparison_medium.csv'
df[['Model', 'Medium HR@10', 'Medium NDCG@10']].rename(
    columns={'Medium HR@10': 'HR@10', 'Medium NDCG@10': 'NDCG@10'}
).to_csv(medium_file, index=False)
print(f"✅ Medium sequence comparison saved: {medium_file}")

# Display results
print("\n" + "="*70)
print("📈 OVERALL PERFORMANCE COMPARISON")
print("="*70)
print(df[['Model', 'HR@10', 'NDCG@10', 'MRR@10']].to_string(index=False))

print("\n" + "="*70)
print("📊 SHORT SEQUENCES (<10 items)")
print("="*70)
print(df[['Model', 'Short HR@10', 'Short NDCG@10']].to_string(index=False))

print("\n" + "="*70)
print("📊 MEDIUM SEQUENCES (10-50 items)")
print("="*70)
print(df[['Model', 'Medium HR@10', 'Medium NDCG@10']].to_string(index=False))

# Calculate improvement/degradation
if len(df) > 0:
    baseline_ndcg = df[df['Model'] == 'bert4rec']['NDCG@10'].values[0]
    
    print("\n" + "="*70)
    print("📉 PERFORMANCE GAP vs BERT4Rec Baseline")
    print("="*70)
    
    for _, row in df.iterrows():
        model = row['Model']
        ndcg = row['NDCG@10']
        
        if model != 'bert4rec':
            diff = ((ndcg - baseline_ndcg) / baseline_ndcg) * 100
            symbol = "📈" if diff > 0 else "📉"
            print(f"{symbol} {model:25s}: {diff:+.2f}% ({ndcg:.6f} vs {baseline_ndcg:.6f})")

print("\n" + "="*70)
print("✅ Analysis Complete!")
print("="*70)

## Step 7: Download Results

Package all results for local analysis

In [ ]:
# Create zip file with all BERT hybrid results
!mkdir -p bert_hybrid_results

# Copy result files
!cp -r results/bert4rec_* bert_hybrid_results/ 2>/dev/null || true
!cp -r results/bert_hybrid_* bert_hybrid_results/ 2>/dev/null || true
!cp results/bert_hybrid_comparison_*.csv bert_hybrid_results/ 2>/dev/null || true

# Create archive
!zip -r bert_hybrid_results.zip bert_hybrid_results/

print("="*70)
print("📦 Results Package Created")
print("="*70)
print("")
print("✅ File: bert_hybrid_results.zip")
print("")
print("Contains:")
print("  - All BERT4Rec experiment folders")
print("  - All BERT4Rec+GNN hybrid experiment folders")
print("  - Comparison CSV files")
print("")
print("Download this file to your local machine for detailed analysis!")
print("="*70)

## 📊 Expected Outcomes

Based on our previous findings with SASRec+GNN hybrids:

**Hypothesis 1: GNN helps BERT4Rec**
- BERT4Rec+GNN should outperform standalone BERT4Rec
- Graph structure provides complementary information

**Hypothesis 2: GNN hurts BERT4Rec** 
- BERT4Rec+GNN underperforms standalone BERT4Rec
- Similar to SASRec+GNN results (-26% to -31% degradation)
- GNN fusion creates information bottleneck

**Hypothesis 3: Mixed results**
- Some fusion strategies work, others don't
- Length-adaptive fusion shows different patterns

---

## 🔬 Key Questions to Answer

1. **Does GNN improve BERT4Rec?** Compare BERT4Rec vs best hybrid
2. **Which fusion works best?** Compare the 4 fusion strategies
3. **Length-adaptive benefits?** Do hybrids help short vs medium sequences differently?
4. **Comparison with SASRec+GNN**: Do bidirectional transformers benefit more from GNN?

---

## 📝 Notes

- Results will be in `results/` directory
- Each model has: config.json, history.json, results.json
- Training time: ~1-1.5 hours per model with GPU T4
- Models use early stopping (patience=20) to prevent overfitting
